<a href="https://colab.research.google.com/github/m-hironori/learing-keras/blob/master/1_%E3%83%A2%E3%83%87%E3%83%AB%E4%BD%9C%E6%88%90%E3%81%AE%E5%9F%BA%E6%9C%AC(MNIST%E3%83%87%E3%83%BC%E3%82%BF%E3%81%A7%E6%89%8B%E6%9B%B8%E3%81%8D%E6%96%87%E5%AD%97%E8%AA%8D%E8%AD%98).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.5'

# モデル作成の基本(MNISTデータで手書き文字認識)



kerasによるモデル作成の基本として、MNIST手書き文字認識を解いてみましょう。



 

## 問題設定

まずは、問題設定を確認してみましょう。


問題属性 | 問題内容
--- | ---
入力データ形式 | 手書きの数字 ( 0 - 9 ) 画像。28 * 28 の白黒。70,000 枚。
予測すること | 0 - 9 のどれが書いてあるか。多クラス分類問題。 
評価指標 | accuracy
評価方法 | ホールドアウト(訓練 60,000, テスト 10,000)

0 から 9 の手書き数字画像を入力に、何の数字かを当てることです。

この問題は、データセットが用意されていること、
データセットのサイズ小さいこと、
DeppLearningの内容説明のしやすさもあり、
非常によく取り上げられるます。

## データ準備



データの準備をしてみましょう。

In [2]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


keras には、あらかじめ MNIST データを準備する関数が用意されているので、これだけで、データを取ってこれます。

`mnist.loaddata()` 関数は、データを特定のURLからダウンロードし、`.keras`フォルダにおき、numpy行列としてロードしています。データがすでにフォルダにある場合はダウンロードしないようになっています。

* データはテンソル ( Numpy でーたが扱いやすい ) として準備
* 値は [ -1 , -1 ] や [ 0 , 1 ] ぐらいの値にスケーリングしておく
* 特徴量の値の範囲がことなったり、異なる種類のデータを使う場合は、スケールを合わせるためにデータを正規化する 



データの準備では、手元にデータを取ってきたら、ニューラルネットワークに合うように整形しておきましょう。特に値が全く異なる範囲の場合は、正規化しておきましょう。あらかじめ値の正規化したり、正規化の仕組みをニューラルネットワーク内に組み込んでおくこともできます。

先ほどとってきたMNISTのデータ内容を見てみましょう。

In [10]:
print("\tデータ型")
print(f"訓練データ\t{type(train_images)}")
print(f"訓練ラベル\t{type(train_labels)}")
print(f"テストデータ\t{type(test_images)}")
print(f"テストラベル\t{type(test_labels)}")

	データ型
訓練データ	<class 'numpy.ndarray'>
訓練ラベル	<class 'numpy.ndarray'>
テストデータ	<class 'numpy.ndarray'>
テストラベル	<class 'numpy.ndarray'>


このように、ロードしたデータは、すでにNumpy形式になっていますね。

In [21]:
import numpy as np

print("\tデータサイズ\t型\t最大値\t最小値")
print(f"訓練データ\t{train_images.shape}\t{train_images.dtype}\t{train_images.max()}\t{train_images.min()}")
print(f"訓練ラベル\t{train_labels.shape}\t{train_labels.dtype}\t{train_labels.max()}\t{train_labels.min()}")
print(f"テストデータ\t{test_images.shape}\t{test_images.dtype}\t{test_images.max()}\t{test_images.min()}")
print(f"テストラベル\t{test_labels.shape}\t{test_labels.dtype}\t{test_labels.max()}\t{test_labels.min()}")

	データサイズ	型	最大値	最小値
訓練データ	(60000, 28, 28)	uint8	255	0
訓練ラベル	(60000,)	uint8	9	0
テストデータ	(10000, 28, 28)	uint8	255	0
テストラベル	(10000,)	uint8	9	0


訓練データは 60000 枚の サイズ 28 * 28 の画像、各値は整数値、最大値 255, 最小値 0です。
訓練ラベルは、60000個の正解データで整数値、最大値 9, 最小値 0です。テストデータは、10000 枚で、同じ形式の画像です。

データは、白黒画像で255諧調の画像ですね。ラベルは、手書き文字のの正解の数字の 0 - 9 ですね。

データは、0 - 1 にスケーリングしておいたほうがよさそうです。

ラベルは、one-hot encoding しておいたほうがよさそうです。

次は、データの中身を一つ取り出して見てみましょう。

	データmax,min
訓練データ	255	0
訓練ラベル	9	0
テストデータ	(10000, 28, 28)	uint8
テストラベル	(10000,)	uint8


In [5]:
train_labels

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)